# PromptGPT - Practical guide

*Could a Large Language Model (LLM) handle the role of Prompt engineer?*

Recently, Large Language Models (think GPT-4, etc) are beginning to beat benchmarks, which are difficult even for humans, such as official exams.

In this guide, I will introduce with a tutorial on: How to get started with PromptGPT. 

## PromptGPT

Prompt GPT includes different types of re-usable Prompt Programs:

1. A single prompt function to make all API calls to a LLM (GPT-4, Anthropic API, etc.) 
2. High-level "cognitive functions" to manage workflow of the PromptGPT, such as: Planning, Tasking, Execution, etc. 
3. "Low level" programs, such as saving & retrieving API calls / memories. 

PromptGPT Settings enable:
* Define instructions for Prompt programs
* Control safety: limit usage and prohibited tasks.

## PromptGPT: Getting started

Let's start by importing libraries. Make sure to add your token/key as an environmental variable

In [64]:
!pip install --upgrade openai
import openai
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
openai.api_key = user_secrets.get_secret("OPENAI_API_KEY")

## PromptGPT: Settings

PromptGPT Settings make it easy to steer PromptGPT behaviour. Each cognitive task uses its own function, which each use their own User Prompt and LLM system instruction. For example planning task is given different instruction, than one executing specific planned action. This enables functions to always use single function for the API call. This simplifies the overall Prompt Programming.

Aspects to define in Settings:
* User input method
* Context / Message lengths
* Model(s) to use
* Memory (filename)
* Instructions: Prompt 
* System instructions: LLMs system instruction on how to handle the Prompt.


In [65]:
# SYSTEM SETTTINGS
#prompt = input("Ask a question: ")
prompt = "Generate 1000 followers with zero budget." #prompt = "Organize and execute a plan to generate 1000 followers in twitter without spending money in 5 steps."
max_length_input = 500
execution_length = 2000
model_to_use = 'gpt-4'
#model_to_use = 'gpt-3.5-turbo'
filepath = 'short_memory.txt'
system_instruction = "You are a world-class prompt engineer producing right prompts to LLM in JSON format only to support programming anything. Only one overall objective. Prompts solve correctly anything by mapping up to 5, short, precise, measurable task/sub-task descriptions, task output types & task role to perform them & necessary concrete resource and/or memory name & use only status messages: Completed, In progress, Error, Rework. Use only single world to define the resource: for example Twitter, Github, Medium. Add the task description specific output type."
define_task = 'Print only the task name from the json: {}.'
define_role = "Print only the task role from the json: {}."
define_resource = "Print only the task resource from the json: {}."
define_output_type = "Print only the task resource from the json: {}."
define_memory = "Print only the task memory from the json: {}."
define_task_system = f"You are a task selector. You only print name of the task. You only pick one task, which is in the first task not completed yet and in status in progresss."
define_role_system = f"You are a role selector. You only print name of the role of the task. You only pick one role, which is in the first task not completed yet and in status in progresss." 
define_resource_system = f"You are a resource selector. You only print resource of the task. You only pick one resource, which is in the first task not completed yet and in status in progresss." 
define_output_type_system = f"You are an output_type selector. You only print output_type of the task. You only pick one output_type, which is in the first task not completed yet and in status in progresss." 
define_memory_system = f"You are a task memory selector. You only print memory name of the task. You only pick one memory, which is in the first task not completed yet and in status in progresss. In case no memory required: print No memory"
execution_script = f"print only the solution as a json:"
execution_system_instruction =  "You are a world-class {} creating a specific answer to: {}. You use as resource: {} to produce {}. Limit is 300 characters" 

**Prompt Programs**

We are now in the section, where the Prompt Programs reside. These are simply Python functions. The first function is the generic LLM API call. It works on any request, which makes simple to integrate inside any other function with specific type of cognitive purpose. 

* Generic gpt_program()-function handles all API calls to a LLM.
* Cognitive skills-functions (Planning, Executing etc) steer the "high-level" workflow
* Specialized skills handle resources: like managing memory

In [66]:
def gpt_program(gpt_program_input, gpt_program_system, gpt_model, max_length):
    """
        ---Generic GPT Program---
        gpt_program_input: what is the input?
        gpt_program_system: What should gpt do?
        gpt_model: What model to use?
        max_length: What is the total length input + output?
    """
    gpt_program_response = openai.ChatCompletion.create(
        model = gpt_model,
        messages = [
            {"role": "system", "content": f"{gpt_program_system}"},
            {"role": "user", "content": ""},
            {"role": "assistant", "content": "How can I help you?"},
            {"role": "user", "content": f"{gpt_program_input}"},
        ],
        max_tokens=max_length,
        temperature=0)
    gpt_program_output = gpt_program_response['choices'][0]['message']['content']
    return gpt_program_output


def save_memory(memory_input, memory_file):
    """
    ---SAVE MEMORY---
    memory_input: What is text to be stored in the .txt-file?
    memory_file: What variable is used to store the text?
    """
    with open(memory_file, 'w') as f:
        f.write(str(memory_input))
    return memory_file


def get_memory(memory_file, memory):
    """
    ---RETRIEVE MEMORY---
    memory_file: What variable is used to store the text?
    memory: What is the memory variable? 
    """
    with open(memory_file, "r") as f:
        memory = f.read()
    return memory 


def get_task(define_task, define_task_system):
    """
    ---GET TASK DESCRIPTION---
    """
    actual_task = ""
    define_task = define_task.format(memory)
    actual_task = gpt_program(define_task, define_task_system, model_to_use, execution_length)
    return actual_task

def get_role(define_role, define_role_system):
    """
    ---GET TASK ROLE---
    """
    actual_task_role = ""
    define_role = define_role.format(memory)
    actual_task_role = gpt_program(define_role, define_role_system, model_to_use, execution_length)
    return actual_task_role

def get_resource(define_resource, define_resource_system):
    """
    ---GET TASK RESOURCE---
    """
    actual_task_resource = ""
    define_resource = define_resource.format(memory)
    actual_task_resource = gpt_program(define_resource, define_resource_system, model_to_use, execution_length)
    return actual_task_resource

def get_output_type(define_output_type, define_output_type_system):
    """
    ---GET TASK OUTPUT TYPE---
    """
    actual_task_output_type = ""
    define_output_type = define_output_type.format(memory)
    actual_task_output_type = gpt_program(define_resource, define_output_type_system, model_to_use, execution_length)
    return actual_task_output_type

def get_task_memory(define_memory, define_memory_system):
    """
    ---GET TASK MEMORY---
    """
    actual_task_memory = ""
    define_memory = define_memory.format(memory)
    actual_task_memory = gpt_program(define_memory, define_memory_system, model_to_use, execution_length)
    return actual_task_memory

def execute(execution_script, execution_system_instruction):
    """
    ---EXECUTE A TASK---
    """
    execution_output = ""
    execution_system_instruction= execution_system_instruction.format(task_role, task, task_resource, task_output_type)
    execution_output = gpt_program(execution_script, execution_system_instruction, model_to_use, execution_length)
    return execution_output


**Planning**

Let's now process the user input. Planning is first task. It uses the generic gpt_program()-function, but adds the Planning specific Instructions & System messages. The result is an execution plan for the user objective.

In [67]:
my_plan = gpt_program(prompt, system_instruction, model_to_use, max_length_input)

**Memory**

I can now store this plan in a memory. We can use it as a short-term memory or as a long-term memory. The difference is token consumption. I could for example store information about a book permanently, but I may not want to store longer a specific plan for completing a specific task. I see this especially impacting to overall token consumption.

Let's finally retrieve the memory from the short_memory.txt file and print it. We use JSON-format, which includes information such as:
- Overall objective
- Task list
- Task description
- Task role
- Task resource
- Task status

These different elements are useful: some include pre-defined options: In progress, Closed/Error/Rework, while for example Role and Task description is defined by LLM.

In [68]:
save_memory(my_plan, filepath)
memory = get_memory(filepath, '')
print(memory)

{
  "objective": "Generate 1000 followers with zero budget",
  "tasks": [
    {
      "description": "Research effective strategies for organic growth on social media platforms",
      "output_type": "List of strategies",
      "role": "Researcher",
      "resource": "Internet",
      "status": "In progress"
    },
    {
      "description": "Create high-quality, engaging content tailored to the target audience",
      "output_type": "Content",
      "role": "Content Creator",
      "resource": "Creativity",
      "status": "In progress"
    },
    {
      "description": "Optimize social media profiles for discoverability and engagement",
      "output_type": "Optimized profiles",
      "role": "Social Media Manager",
      "resource": "Social Media Platforms",
      "status": "In progress"
    },
    {
      "description": "Implement growth strategies and monitor progress",
      "output_type": "Growth metrics",
      "role": "Social Media Manager",
      "resource": "Analytics",
    

**Task definition**

PromptGPT uses pre-defined functions to look up from the overall plan, the first available task not yet completed. It will then retrieve the respective task description, role and resources for it. This way, makes the PromptGPT very autonomous. I can simply send it to look up the current task list and it can figure out right parameters for executing it.

In [69]:
task = get_task(define_task, define_task_system)
task_role = get_role(define_role, define_role_system)
task_resource = get_resource(define_resource, define_resource_system)
task_output_type = get_output_type(define_output_type, define_output_type_system) 
#task_memory = get_memory(define_memory, define_memory_system)
print(task + "/ " + task_role + "/ " + task_resource + "/ " + task_output_type)

Research effective strategies for organic growth on social media platforms/ Researcher/ Internet/ Please provide the JSON data containing the task resource and its status, so I can help you with your request.


**Execution**

PromptGPT executes tasks using the task description, task role and task resources defined in the previous step. The

In [70]:
# EXECUTION
executed_json = execute(execution_script, execution_system_instruction)
print(executed_json)

{
  "strategies": [
    {
      "name": "Consistent Posting",
      "description": "Post regularly to maintain audience engagement and increase visibility."
    },
    {
      "name": "Engaging Content",
      "description": "Create high-quality, relevant, and engaging content that resonates with your target audience."
    },
    {
      "name": "Hashtags",
      "description": "Use relevant and trending hashtags to increase the discoverability of your content."
    },
    {
      "name": "Collaborations",
      "description": "Partner with influencers and other brands to expand your reach and credibility."
    },
    {
      "name": "User-Generated Content",
      "description": "Encourage and share content created by your audience to foster community and trust."
    },
    {
      "name": "Contests and Giveaways",
      "description": "Host contests and giveaways to increase engagement and attract new followers."
    },
    {
      "name": "Analytics",
      "description": "Monitor a